In [1]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# %matplotlib inline

plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import mutual_info_classif

import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_addons as tfa

import transformers
from transformers import AutoTokenizer
from transformers import TFAutoModel
transformers.logging.set_verbosity_error()

import re
from glob import glob
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

import argparse
# import wandb
# from wandb.keras import WandbCallback
# wandb.init(project="DACON_235978", name="gnoeyheat")

parser = argparse.ArgumentParser(description="gnoeyheat")
parser.add_argument('--text_pretrained_model', default="roberta", type=str)
parser.add_argument('--text_len', default=300, type=int)
parser.add_argument('--optimizer', default="sgd", type=str) # sgd or adam
parser.add_argument('--learning_rate', default=0.002, type=float)
parser.add_argument('--loss', default='cc', type=str) # cc or fl
parser.add_argument('--label_smoothing', default=0.1, type=float)
parser.add_argument('--batch_size', default=16, type=int)
parser.add_argument('--epochs', default=30, type=int)
parser.add_argument('--validation_split', default=0.1, type=float)
parser.add_argument('--seed', default=1011, type=int)
args = parser.parse_args('')

# wandb.config.update(args)

os.environ["CUDA_VISIBLE_DEVICES"]="0"

text_pretrained_model = args.text_pretrained_model
text_len = args.text_len
BATCH_SIZE=args.batch_size
EPOCHS=args.epochs
VALIDATION_SPLIT=args.validation_split
SEED=args.seed

def set_seeds(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_seeds()

In [2]:
if args.text_pretrained_model == "roberta":
    text_pretrained_model = "klue/roberta-large"

tokenizer = AutoTokenizer.from_pretrained(text_pretrained_model)
# tokenizer.truncation_side = 'left'

In [3]:
train = pd.read_csv("./input/train.csv")
test = pd.read_csv("./input/test.csv")

X_txt = train["overview"]
X_test_txt = test["overview"]

y = train["cat3"]
y_encoder = {key : value for key, value in zip(np.unique(y), range(len(np.unique(y))))}
y = np.array([y_encoder[k] for k in y])

X_txt.shape, X_test_txt.shape, y.shape

((16986,), (7280,), (16986,))

In [4]:
def text_cleaning(df):
    df = df.apply(lambda x : re.sub('[^ ㄱ-ㅣ가-힣]+', ' ', x))
    df = df.apply(lambda x : ' '.join(x.split()))
    return df

X_txt = text_cleaning(X_txt)
X_test_txt = text_cleaning(X_test_txt)

X_txt.iloc[0]

'소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 올리고 있으며 바다낚시터로도 유명하다 항 주변에 설치된 양식장들은 섬사람들의 부지런한 생활상을 고스 란히 담고 있으며 일몰 때 섬의 정경은 바다의 아름다움을 그대로 품고 있는 듯하다 또한 섬에는 각시여 전설 도둑바위 등의 설화가 전해 내려오고 있으며 매년 정월 풍어제 풍속이 이어지고 있다'

In [5]:
train["len"] = train["overview"].apply(tokenizer.tokenize).apply(len)
test["len"] = test["overview"].apply(tokenizer.tokenize).apply(len)
train["len"].median(), test["len"].median()

(144.0, 144.0)

In [6]:
class DataGenerator(tf.keras.utils.Sequence):

    def __init__(
        self,
        sentence,
        labels,
        batch_size=BATCH_SIZE,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence = sentence
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        self.tokenizer = tokenizer
        self.indexes = np.arange(len(self.sentence))
        self.on_epoch_end()

    def __len__(self):
        return len(self.sentence) // self.batch_size

    def __getitem__(self, idx):
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence = self.sentence[indexes]

        encoded = self.tokenizer.batch_encode_plus(
            sentence.tolist(),
            add_special_tokens=True,
            padding="max_length",
            truncation=True,
            max_length=text_len,
            return_tensors="tf",
            return_token_type_ids=True,
            return_attention_mask=True,
        )

        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        if self.shuffle:
            np.random.RandomState(SEED).shuffle(self.indexes)

In [7]:
X_train, X_val, y_train, y_val = train_test_split(X_txt, y, test_size=VALIDATION_SPLIT, random_state=SEED, stratify=y)
y_train = tf.keras.utils.to_categorical(y_train)
y_val = tf.keras.utils.to_categorical(y_val)

X_train.shape, X_val.shape, y_train.shape, y_val.shape

((15287,), (1699,), (15287, 128), (1699, 128))

In [8]:
train_ds = DataGenerator(
    X_train.values, y_train,
    batch_size=BATCH_SIZE,
    shuffle=True,
)
val_ds = DataGenerator(
    X_val.values, y_val,
    batch_size=BATCH_SIZE,
    shuffle=False,
)

lr = 2e-2

In [9]:
input_ids = tf.keras.layers.Input(
    shape=(text_len,), dtype=tf.int32, name="input_ids"
)
attention_masks = tf.keras.layers.Input(
    shape=(text_len,), dtype=tf.int32, name="attention_masks"
)
token_type_ids = tf.keras.layers.Input(
    shape=(text_len,), dtype=tf.int32, name="token_type_ids"
)

bert_model = TFAutoModel.from_pretrained(text_pretrained_model, from_pt=True)
bert_model.trainable = True

bert_output = bert_model(
    input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
)

x = bert_output.last_hidden_state
x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.2)(x)

output = layers.Dense(y_train.shape[1], activation="softmax")(x)

model = tf.keras.models.Model(
    inputs=[input_ids, attention_masks, token_type_ids], outputs=output
)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [10]:
if args.optimizer == "sgd":
    optim = tf.keras.optimizers.SGD(
        learning_rate=lr, momentum=0.9
    )
if args.loss == "cc":
    loss_function = tf.keras.losses.CategoricalCrossentropy(
        label_smoothing=args.label_smoothing
    )

model.compile(
    optimizer=optim,
    loss=loss_function,
    metrics=tfa.metrics.F1Score(num_classes=y_train.shape[1], average="weighted")
)
# tf roberta_model 과 마지막 dense layer 에만 가중치가 존재할거고 그걸 학습해야 되지 않을까
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 300)]        0           []                               
                                                                                                  
 attention_masks (InputLayer)   [(None, 300)]        0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, 300)]        0           []                               
                                                                                                  
 tf_roberta_model (TFRobertaMod  TFBaseModelOutputWi  336656384  ['input_ids[0][0]',              
 el)                            thPooling(last_hidd               'attention_masks[0][0]',    

In [11]:
model.loss

In [12]:
# model checkpoint가 없어서 그런 듯한데 , 꼼꼼히 살펴보니까
# checkpoint_path

In [13]:
checkpoint_path = f"load_model/{parser.description}"

def scheduler(epoch, lr):
    if epoch < 20:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

callback = [
    tf.keras.callbacks.LearningRateScheduler(scheduler),
    # tf.keras.callbacks.ModelCheckpoint(
    #     checkpoint_path,
    #     monitor="val_f1_score",
    #     save_best_only=True,
    #     save_weights_only=True,
    #     mode="max",
    # )
]

history = model.fit(
    train_ds,
    epochs=2,
    callbacks=[callback],
    validation_data=val_ds,
)
# 그러기엔 loss가 높다

# train 이 안되는 현상 나랑 똑같다 weight가 reshape 되면 그렇다고 ?
# https://stackoverflow.com/questions/61763029/tensorflow-tf-reshape-causes-gradients-do-not-exist-for-variables
# eager execution 과 관련이 있다..?

Epoch 1/2


ResourceExhaustedError:  OOM when allocating tensor with shape[16,300,1024] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model/tf_roberta_model/roberta/encoder/layer_._5/output/dropout_17/dropout/random_uniform/RandomUniform
 (defined at c:\users\data\appdata\local\programs\python\python38\lib\site-packages\keras\layers\core\dropout.py:105)
]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_50301]

Errors may have originated from an input operation.
Input Source operations connected to node model/tf_roberta_model/roberta/encoder/layer_._5/output/dropout_17/dropout/random_uniform/RandomUniform:
In[0] model/tf_roberta_model/roberta/encoder/layer_._5/output/dropout_17/dropout/Shape:

Operation defined at: (most recent call last)
>>>   File "c:\users\data\ap

In [ ]:
acc = history.history['f1_score']
val_acc = history.history['val_f1_score']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.plot(acc, label='Training Weighted-F1')
plt.plot(val_acc, label='Validation Weighted-F1')
plt.legend(loc='lower right')
plt.title('Training and Validation Weighted-F1')
plt.show()

plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model.load_weights(checkpoint_path)

In [ ]:
val_weighted_f1 = model.evaluate(val_ds)[1]
print(f"val_weighted_f1: {val_weighted_f1}")

In [ ]:
test_ds = DataGenerator(
    X_test_txt.values, None,
    batch_size=1,
    shuffle=False,
    include_targets=False,
)

pred_prob = []
for i in range(test_ds.__len__()):
    pred_prob.append(model.predict(test_ds.__getitem__(i)))
pred_prob = np.vstack(pred_prob)
pred = np.argmax(pred_prob, axis=1)

y_decoder = {value : key for key, value in y_encoder.items()}
result = np.array([y_decoder[v] for v in pred])

pd.Series(result).value_counts()

In [ ]:
submission = pd.read_csv("./sample_submission.csv")
submission["cat3"] = result
submission.to_csv(f"{parser.description}.csv", index=False)